In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
from scipy import stats

In [3]:
df = pd.read_csv('case_study_data.csv')

# Clean the Data

### Missing Data

In [4]:
df.describe()

,search_date,arrival,departure,num_adults,num_children,search_id,hotel_id,hotel_price,is_promo,hotel_feature_1,hotel_feature_2,hotel_feature_3,hotel_feature_4,hotel_feature_5,booked
count,46647.000000,46647.000000,46647.000000,46647.000000,46647.000000,46647.000000,46647.000000,46647.000000,46647.000000,45445.000000,46322.000000,46647.000000,46647.000000,46647.000000,46647.000000
mean,11.734517,126.788046,134.479902,2.370570,0.812335,5509.648659,23095.564302,2599.055621,0.501490,76.849742,63.108847,59.068729,5.982335,2.357601,0.006603
std,7.336887,90.867556,92.517336,0.912667,0.949223,4460.689701,13480.392766,4490.091588,0.500003,41.747830,22.846125,51.378153,6.014773,1.811249,0.080990
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,30.370000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,6.000000,49.000000,54.000000,2.000000,0.000000,1656.000000,11474.000000,559.400000,0.000000,46.238512,52.000000,16.000000,1.000000,1.000000,0.000000
50%,12.000000,109.000000,116.000000,2.000000,0.000000,4488.000000,23309.000000,1336.600000,1.000000,67.712628,52.000000,46.000000,4.000000,1.000000,0.000000
75%,17.000000,187.000000,197.000000,3.000000,2.000000,8727.000000,33668.000000,3090.510000,1.000000,128.117134,90.000000,86.000000,9.000000,4.000000,0.000000
max,25.000000,493.000000,501.000000,8.000000,6.000000,16015.000000,46647.000000,175454.410000,1.000000,143.777606,100.000000,287.000000,36.000000,7.000000,1.000000


We can see from the count of each column that 'hotel_feature_1' and 'hotel_feature_2' are missing values

In [5]:
print "Number of missing hotel_feature_1 entries: " + str(len(df[np.isnan(df['hotel_feature_1'])]))
print "Number of missing hotel_feature_2 entries: " + str(len(df[np.isnan(df['hotel_feature_2'])]))

Number of missing hotel_feature_1 entries: 1202
Number of missing hotel_feature_2 entries: 325


In [79]:
df.reset_index('hotel_id').head()

,index,search_date,arrival,departure,num_adults,num_children,search_id,hotel_id,hotel_price,is_promo,hotel_feature_1,hotel_feature_2,hotel_feature_3,hotel_feature_4,hotel_feature_5,booked
0,0,25,457,471,3,2,0,517,1724.38,0,64.490309,85.0,7,0,0,0
1,1,25,457,471,3,2,0,517,1724.38,0,64.490309,85.0,7,0,0,0
2,2,25,457,471,3,2,0,29771,1905.54,1,25.826343,52.0,13,1,1,0
3,3,25,458,463,2,0,1,517,2077.95,0,64.490309,85.0,9,0,0,0
4,4,25,458,463,2,0,1,40744,6822.10,0,24.959968,52.0,11,1,2,0


#### Hotel Feature 1

Check to see if all unique 'hotel_id' rows are paired with the same unique 'hotel_feature_1' e.g. hotel_id=517 will always have hotel_feature_1=64.490309 etc.

In [37]:
df_nonan_feature1 = df[np.isnan(df['hotel_feature_1']) != True]
hotel_id_feature1_pairs_count = df_nonan_feature1.groupby('hotel_id').agg({'hotel_feature_1': pd.Series.nunique})
max(hotel_id_feature1_pairs_count['hotel_feature_1'])

1.0

Since we can see that they are paired, we can replace missing values of hotel_feature_1 with a value based on their 'hotel_id'

In [38]:
hotel_id_feature1_pairs = df_nonan_feature1.groupby('hotel_id').agg({'hotel_feature_1':'first'})

In [78]:
hotel_id_feature1_pairs.reset_index('hotel_id').head()

,hotel_id,hotel_feature_1
0,1,63.725976
1,116,75.379042
2,175,52.820451
3,297,128.856509
4,443,120.440857


Now we have a series of pairs of 'hotel_id' to 'hotel_feature_1' we can insert the missing information into the series

In [60]:
hotel_id_feature1_pairs.loc[116]['hotel_feature_1'].item()

75.3790421802

In [68]:
def get_hotel_feature_1(hotel_id):
    return hotel_id_feature1_pairs.loc[hotel_id]['hotel_feature_1'].item()


#df.apply(lambda row: hotel_id_feature1_pairs.loc[row['hotel_id']]['hotel_feature_1'].item())
replace_nan_with_hotel_feature_1(116, hotel_id_feature1_pairs)

75.3790421802

In [69]:
df['hotel_id'][0]

517

# ** I Can't Figure Out How To Do This So I'm Deleting The Rows

In [85]:
df = df[np.isnan(df['hotel_feature_1']) != True]
df = df[np.isnan(df['hotel_feature_2']) != True]

In [86]:
df.head()

,search_date,arrival,departure,num_adults,num_children,search_id,hotel_id,hotel_price,is_promo,hotel_feature_1,hotel_feature_2,hotel_feature_3,hotel_feature_4,hotel_feature_5,booked
0,25,457,471,3,2,0,517,1724.38,0,64.490309,85.0,7,0,0,0
1,25,457,471,3,2,0,517,1724.38,0,64.490309,85.0,7,0,0,0
2,25,457,471,3,2,0,29771,1905.54,1,25.826343,52.0,13,1,1,0
3,25,458,463,2,0,1,517,2077.95,0,64.490309,85.0,9,0,0,0
4,25,458,463,2,0,1,40744,6822.10,0,24.959968,52.0,11,1,2,0
